In [1]:
import requests
import pandas as pd
import numpy as np
from pprint import pprint
from config import census_key
import json

In [2]:
year_list = ['2009', '2010','2011', '2012', '2013', '2014', '2015', '2016']
fail_master = []

fuck_you_census = {
    '2009': '',
    '2010': 'acs/',
    '2011': 'acs/',
    '2012': 'acs/',
    '2013': 'acs/',
    '2014': 'acs/',
    '2015': 'acs/',
    '2016': 'acs/'
}


variable_conversion = {
    '2009': ['B19013_001E', 'B25064_001E'],
    '2010': ['B19013_001E', 'B25064_001E'],
    '2011': ['B19013_001E', 'B25064_001E'],
    '2012': ['B19013_001E', 'B25064_001E'],
    '2013': ['B19013_001E', 'B25064_001E'],
    '2014': ['B19013_001E', 'B25064_001E'],
    '2015': ['B19013_001E', 'B25031_001E'],
    '2016': ['B19013_001E', 'B25031_001E']
}

In [5]:
census_key

'3cca80dd17030630a680da580efab47d1f498bf7'

In [15]:
for year in year_list:
    try:
        census_test = requests.get(f'https://api.census.gov/data/{year}/acs/flows?get=FULL1_NAME,FULL2_NAME,MOVEDIN,MOVEDOUT,MOVEDNET' +
                                       f'&for=county:*&in=state:06&key=' + census_key).json()
        census_df = pd.DataFrame(census_test)
        census_df.columns = census_df.iloc[0]
        census_df = census_df[1:]
        census_df = census_df.rename(columns = {'FULL1_NAME': 'County',
                                                'MOVEDIN': f'{year}IN',
                                                'MOVEDOUT': f'{year}OUT',
                                                'MOVEDNET': f'{year}NET'
                                                })
        census_df = census_df[['County', f'{year}IN', f'{year}OUT', f'{year}NET']]
        
        economic_test = requests.get(f'https://api.census.gov/data/{year}/{fuck_you_census[year]}acs5?get=NAME,' + 
                                     f'{variable_conversion[year][0]},' +
                                     f'{variable_conversion[year][1]}' +
                                        '&for=county:*&in=state:06&key=' +
                                        census_key).json()
        economic_df = pd.DataFrame(economic_test)
        economic_df.columns = economic_df.iloc[0]
        economic_df = economic_df[1:]
        economic_df = economic_df.rename(columns = {f'{variable_conversion[year][0]}': f'{year}MED_INC',
                                                f'{variable_conversion[year][1]}': f'{year}RENT'
                                                })
        economic_df = economic_df[['NAME', f'{year}MED_INC', f'{year}RENT']]
        
        if year == '2009':
            census_master = census_df
            economic_master = economic_df
            
        else:
            census_master = census_master.merge(census_df, on = 'County')
            economic_master = economic_master.merge(economic_df, left_on = 'NAME', right_on = 'NAME')
    
    except:
        fail_master.append(year)
        print(year+" FAILED")

2011 FAILED
2012 FAILED
2013 FAILED
2014 FAILED
2015 FAILED
2016 FAILED


In [6]:
census_test = requests.get(f'https://api.census.gov/data/{year_list[2]}/acs/flows?get=FULL1_NAME,FULL2_NAME,MOVEDIN,MOVEDOUT,MOVEDNET,POP1YR' +
                                       f'&for=county:*&in=state:06&key=' + census_key).json()
census_df = pd.DataFrame(census_test)
census_df.columns = census_df.iloc[0]
census_df = census_df[1:]
census_df = census_df.rename(columns = {'FULL1_NAME': 'County',
                                        'MOVEDIN': f'{year_list[2]}IN',
                                        'MOVEDOUT': f'{year_list[2]}OUT',
                                        'MOVEDNET': f'{year_list[2]}NET',
                                        'POP1YR':f'{year_list[2]}POP1YR'
                                        })
census_df = census_df[['County', f'{year_list[2]}IN', f'{year_list[2]}OUT', f'{year_list[2]}NET', f'{year_list[2]}POP1YR']]

In [ ]:
economic_test = requests.get(f'https://api.census.gov/data/{year_list[2]}/{fuck_you_census[year_list[2]]}acs5?get=NAME,' + 
                                     f'{variable_conversion[year_list[2]][0]},' +
                                     f'{variable_conversion[year_list[2]][1]}' +
                                        '&for=county:*&in=state:06&key=' +
                                        census_key).json()
economic_df = pd.DataFrame(economic_test)
economic_df.columns = economic_df.iloc[0]
economic_df = economic_df[1:]
economic_df = economic_df.rename(columns = {f'{variable_conversion[year_list[2]][0]}': f'{year_list[2]}MED_INC',
                                        f'{variable_conversion[year_list[2]][1]}': f'{year_list[2]}RENT',
                                        })
economic_df = economic_df[['NAME', f'{year_list[2]}MED_INC', f'{year_list[2]}RENT']]

In [7]:
economic_test = requests.get(f'https://api.census.gov/data/{year_list[0]}/{fuck_you_census[year_list[0]]}acs5?get=NAME,' + 
                                     f'{variable_conversion[year_list[0]][0]},' +
                                     f'{variable_conversion[year_list[0]][1]}' +
                                        '&for=county:*&in=state:06&key=' +
                                        census_key).json()
flamflam = pd.DataFrame(economic_test)
flamflam.columns = flamflam.iloc[0]
flamflam = flamflam[1:]
flamflam = flamflam.rename(columns = {f'{variable_conversion[year_list[0]][0]}': f'{year_list[0]}MED_INC',
                                        f'{variable_conversion[year_list[0]][1]}': f'{year_list[0]}RENT',
                                        })
flamflam = flamflam[['NAME', f'{year_list[0]}MED_INC', f'{year_list[0]}RENT']]

In [9]:
flimflim = requests.get(f'https://api.census.gov/data/{year_list[0]}/acs/flows?get=FULL1_NAME,FULL2_NAME,MOVEDIN,MOVEDOUT,MOVEDNET,POP1YR' +
                                       f'&for=county:*&in=state:06&key=' + census_key).json()
flimflim = pd.DataFrame(flimflim)
flimflim.columns = flimflim.iloc[0]
flimflim = flimflim[1:]
flimflim = flimflim.rename(columns = {'FULL1_NAME': 'County',
                                        'MOVEDIN': f'{year_list[0]}IN',
                                        'MOVEDOUT': f'{year_list[0]}OUT',
                                        'MOVEDNET': f'{year_list[0]}NET',
                                        'POP1YR':f'{year_list[0]}POP1YR'
                                        })
flimflim = flimflim[['County', f'{year_list[0]}IN', f'{year_list[0]}OUT', f'{year_list[0]}NET', f'{year_list[0]}POP1YR']]

In [17]:
census_master = flimflim.merge(census_df, on = 'County')
type(census_master['County'][1])

str

In [14]:
econ_m = flamflam.merge(economic_df, left_on = 'NAME', right_on = 'NAME')
econ_m

,NAME,2009MED_INC,2009RENT,2011MED_INC,2011RENT
0,"Alameda County, California",68863,1166,70821,1228
1,"Alpine County, California",54605,820,59018,796
2,"Amador County, California",57347,1075,56180,1011
3,"Butte County, California",42002,843,42971,878
4,"Calaveras County, California",57676,920,55256,1031
5,"Colusa County, California",47150,762,49558,885
6,"Contra Costa County, California",77838,1239,79135,1309
7,"Del Norte County, California",38408,770,37588,869
8,"El Dorado County, California",70765,1062,68815,1098
9,"Fresno County, California",46230,801,46903,850


In [ ]:
merged = census_df.merge(flimflim, on = 'County')
merged.fillna(0)
merged_group = merged.groupby('County').sum()

In [ ]:
merged_group

In [ ]:
census_final = census_master.groupby(census_master["County"]).sum()

In [ ]:
census_master

In [39]:
for year  in year_list:
    print(year)
    economic_test = requests.get(f'https://api.census.gov/data/{year}/' + 
                                 f'{fuck_you_census[year]}' +
                                'acs5?get=NAME,' + 
                                f'{variable_conversion[year][0]},' +
                                f'{variable_conversion[year][1]}' +
                                '&for=county:*&in=state:06&key=' +
                                census_key).json()
    economic_df = pd.DataFrame(economic_test)
    economic_df.columns = economic_df.iloc[0]
    economic_df = economic_df[1:]
    
    if year == '2009':
        economic_master = economic_df
    else:
        economic_master = pd.concat([economic_master, economic_df])

2009
2010
2011
2012
2013
2014
2015


C:\Users\samue\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



2016
